# Reconciliation Monthly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
from datetime import datetime, timedelta
from config import Config

%load_ext sql
%config SqlMagic.displaylimit = 5

# Parameters cell for external parameters via papermill (job running this notebook will insert a parameter cell below this). This cell has a tag of with the name "parameters" that is used by papermill

e.g.
param1 = "some_value"

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
%sql $Config.SQLALCHEMY_DATABASE_URI

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql
select now() AT TIME ZONE 'PST' as current_date

Query ...

In [ ]:
%%sql monthly_reconciliation_summary  <<
SELECT id, (created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date,  total,  service_fees, payment_method_code, corp_type_code, created_by, payment_date
FROM invoices
WHERE corp_type_code = 'CSO'
AND total > 0
AND invoice_status_code = 'PAID'
AND payment_method_code in ('PAD','EJV')
AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  > (current_date - 1 - interval '1 months')::date
AND created_on AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver' <= (current_date - 1)::date
order by 1;

Save to CSV

In [ ]:
filename_summary = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_summary_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_summary = monthly_reconciliation_summary.DataFrame()
with open(filename_summary, 'w') as f:
    f.write('Monthly Reconciliation Summary:\n\n')
    if df_summary.empty:
        f.write('No Data Retrieved')
    else:
        df_summary.to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql monthly_reconciliation_disbursed  <<
SELECT id, (disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver')::date, total, service_fees, payment_method_code, corp_type_code,created_by
FROM invoices
WHERE corp_type_code = 'CSO'
AND invoice_status_code = 'PAID'
AND payment_method_code in ('PAD','EJV')
AND disbursement_status_code = 'COMPLETED'
AND disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  > (current_date - 1 - interval '1 months')::date
AND disbursement_date AT TIME ZONE 'UTC' AT TIME ZONE 'America/Vancouver'  <= (current_date - 1)::date
order by 1;

Save to another CSV

In [ ]:
filename_disbursed = os.path.join(os.getcwd(), r'data/')+partner_code+'_monthly_reconciliation_disbursed_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m') +'.csv'
df_disbursed = monthly_reconciliation_disbursed.DataFrame()
with open(filename_disbursed, 'a') as f:
    f.write('Monthly Reconciliation Disbursed:\n\n')
    if df_disbursed.empty:
        f.write('No Data Retrieved')
    else:
        df_disbursed.to_csv(f, sep=',', encoding='utf-8', index=False)
